In [1]:
from pathlib import Path

import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tardis.io.atom_data import AtomData
from tardis.io.configuration.config_reader import Configuration

home = Path.home()

ION_SLICE = (1, slice(None), slice(None), slice(None))

# identical atomic data to that used by C Vogl
atom_data = AtomData.from_hdf(
    #"/storage/shield90/merged_mod_20SNG_forbidden_yg_fix_H30_cmfgen_yg_CONVERTED.h5"
    "/storage/mcconnor/christians_atomdata_converted_04Dec25.h5"
)  # currently not available for public use

atom_data.prepare_atom_data([1], "macroatom", [(1, 0)], [(1, 0)])

config = Configuration.from_yaml(
    home / "tardis/tardis/plasma/tests/data/plasma_base_test_config.yml"
)

config.supernova.time_explosion = 16.084 * u.day
config.model.structure.type = "file"
config.model.structure.filename = (
    home
    / "tardis/docs/physics/plasma/equilibrium/cmfgen_stephane_density_rebin.dat"
)
config.model.structure.filetype = "simple_ascii"
config.model.structure.v_inner_boundary = 10000 * u.km / u.s
config.model.structure.v_outer_boundary = 15000 * u.km / u.s

config.model.abundances.He = 0
config.model.abundances.H = 1

config.plasma.excitation = "dilute-lte"
config.plasma.ionization = "nebular"

config.plasma.continuum_interaction.species = ["H 1"]
config.plasma.nlte.species = [(1, 0)]
config.plasma.nlte_ionization_species = ["H 1"]
config.plasma.nlte_excitation_species = ["H 1"]

/home/afullard/tardis/tardis/__init__.py:17: UserWarning: Astropy is already imported externally. Astropy should be imported after TARDIS.
  warnings.warn(


In [2]:
time_simulation = 7.2671371e-44 * u.s
volume = 1.61751052e44 * np.ones(1) * u.cm**3

In [3]:
elemental_number_density = pd.DataFrame(2206918615.4642744 * np.ones(1), index=[1])
elemental_number_density.index.name = "atomic_number"

# Set up atomic data for compatibility

In [4]:
from tardis.iip_plasma.continuum.base_continuum_data import ContinuumData

atom_data.continuum_data = ContinuumData(
               atom_data, selected_continuum_species=[(1, 0)]
           )

atom_data.yg_data.columns = list(atom_data.collision_data_temperatures)

atom_data.nlte_data._init_indices()

atom_data.has_collision_data = False

### Set up plasma

In [5]:
from tardis.iip_plasma.standard_plasmas import LegacyPlasmaArray

plasma = LegacyPlasmaArray(
    elemental_number_density,
    atom_data,
    config.supernova.time_explosion.to("s").value,
    nlte_config=config.plasma.nlte,
    delta_treatment=None,
    ionization_mode="nlte",
    excitation_mode="dilute-lte",
    line_interaction_type=config.plasma.line_interaction_type,
    link_t_rad_t_electron=1.0,
    # link_t_rad_t_electron=self.ws**0.25,
    helium_treatment="none",
    heating_rate_data_file=None,
    v_inner=None,
    v_outer=None,
    continuum_treatment=True,
)

Zeta_data missing - replaced with 1s. Missing ions: []


### Set up radiation field

First plasma solution BEFORE MC step

In [6]:
plasma.ion_number_density

0
atomic_number ion_number              
1             0           3.972527e+04
              1           2.206879e+09

In [7]:
plasma.beta_sobolev

0
atomic_number ion_number level_number_lower level_number_upper          
1             0          0                  29                  0.695675
                                            28                  0.671043
                                            27                  0.644286
                                            26                  0.615253
                                            25                  0.583914
...                                                                  ...
                         24                 25                  0.020919
                         25                 26                  0.018674
                         26                 27                  0.016735
                         27                 28                  0.015054
                         28                 29                  0.013591

[435 rows x 1 columns]

In [8]:
j_blues_ctardis = pd.read_csv(
    "/home/afullard/tardis-chvogl-configs/j_blues_first.csv", index_col=0
)

radiation_temp = 9984.96131287 * np.ones(1)
dilution_factor = 0.1863524378417558 * np.ones(1)

In [9]:
plasma.update_radiationfield(
            radiation_temp, dilution_factor, j_blues_ctardis["0"],
            config.plasma.nlte, initialize_nlte=True,
            n_e_convergence_threshold=0.05, **{})

In [10]:
plasma.level_boltzmann_factor

0
atomic_number ion_number level_number          
1             0          0             2.000000
                         1             0.000011
                         2             0.000003
                         3             0.000002
                         4             0.000002
                         5             0.000003
                         6             0.000003
                         7             0.000004
                         8             0.000005
                         9             0.000006
                         10            0.000007
                         11            0.000008
                         12            0.000009
                         13            0.000011
                         14            0.000012
                         15            0.000014
                         16            0.000016
                         17            0.000017
                         18            0.000019
                         19            0.000021
                         20            0.000023
                         21            0.000026
                         22            0.000028
                         23            0.000030
                         24            0.000033
                         25            0.000035
                         26            0.000038
                         27            0.000041
                         28            0.000044
                         29            0.000047
              1          0             1.000000

In [11]:
plasma.ion_number_density

0
atomic_number ion_number              
1             0           1.435233e+05
              1           2.206775e+09

In [12]:
plasma.level_number_density.head()

0
atomic_number ion_number level_number               
1             0          0             143486.053493
                         1                  0.761167
                         2                  0.190712
                         3                  0.157256
                         4                  0.172186

In [13]:
plasma.electron_densities

0    2.206775e+09
dtype: float64

In [14]:
plasma.beta_sobolev.head()

0
atomic_number ion_number level_number_lower level_number_upper          
1             0          0                  29                  0.335078
                                            28                  0.307649
                                            27                  0.280623
                                            26                  0.254223
                                            25                  0.228743

In [15]:
plasma.previous_beta_sobolev.head()

0
atomic_number ion_number level_number_lower level_number_upper          
1             0          0                  29                  0.695675
                                            28                  0.671043
                                            27                  0.644286
                                            26                  0.615253
                                            25                  0.583914

# Update plasma following ctardis

Also update radiation field and electron distribution

In [16]:
radiation_temp = 9992.272296952351 * np.ones(1)
dilution_factor = 0.3571996025271447 * np.ones(1)

j_blues_ctardis = pd.read_csv(
    "/home/afullard/tardis-chvogl-configs/j_blues_second.csv", index_col=0
)

In [17]:
photo_ion_estimator = pd.read_csv(
    "/home/afullard/tardis-chvogl-configs/photo_ion_estimator.csv",
    index_col=(0),
)
stim_recomb_estimator = pd.read_csv(
    "/home/afullard/tardis-chvogl-configs/stim_recomb_estimator.csv",
    index_col=(0),
)

photo_ion_estimator.columns = photo_ion_estimator.columns.astype(int)
stim_recomb_estimator.columns = stim_recomb_estimator.columns.astype(int)
# Create MultiIndex for photo_ion_estimator
photo_ion_estimator_idx = pd.MultiIndex.from_tuples(
    [(1, 0, level) for level in photo_ion_estimator.index],
    names=["atomic_number", "ion_number", "level_number"],
)
photo_ion_estimator.index = photo_ion_estimator_idx

# Create MultiIndex for stim_recomb_estimator
stim_recomb_estimator_idx = pd.MultiIndex.from_tuples(
    [(1, 0, level) for level in stim_recomb_estimator.index],
    names=["atomic_number", "ion_number", "level_number"],
)
stim_recomb_estimator.index = stim_recomb_estimator_idx

In [18]:
data_path = home / "tardis-regression-data/testdata/thermal_data"
bf_heating_estimator = pd.read_csv(
    data_path / "thermal_bf_heating_est.csv", index_col=(0, 1, 2)
)
stim_recomb_cooling_estimator = pd.read_csv(
    data_path / "thermal_stim_cooling_est.csv", index_col=(0, 1, 2)
)

stim_recomb_cooling_estimator = pd.read_csv(
    data_path / "thermal_stim_cooling_est.csv", index_col=(0, 1, 2)
)

coll_deexc_heating_estimator = pd.read_csv(
    data_path / "coll_deexc_heating_estimator.csv", index_col=(0)
)


ff_heating_estimator = [
    4.89135279e-24,
    4.37696370e-24,
    3.75869301e-24,
    4.97847160e-24,
    4.52158002e-24,
    4.21024499e-24,
    3.94991540e-24,
    3.72915649e-24,
    3.58902110e-24,
    3.40170224e-24,
    3.20848519e-24,
    3.03540032e-24,
    2.87314722e-24,
    2.74328938e-24,
    2.61063140e-24,
    2.50640248e-24,
    2.38164559e-24,
    2.26967531e-24,
    2.24509826e-24,
    2.12378192e-24,
    2.02063266e-24,
    1.92509873e-24,
    1.83070678e-24,
    1.77346374e-24,
]

# because pandas reads in the columns as strings, we need to convert them back to integers
bf_heating_estimator.columns = bf_heating_estimator.columns.astype(int)
stim_recomb_cooling_estimator.columns = (
    stim_recomb_cooling_estimator.columns.astype(int)
)
coll_deexc_heating_estimator.columns = coll_deexc_heating_estimator.columns.astype(int)


In [19]:
continuum_estimators = {}

continuum_estimators['photo_ion_estimator'] = photo_ion_estimator.loc[:, [0]].values
continuum_estimators['stim_recomb_estimator'] = stim_recomb_estimator.loc[:, [0]].values
continuum_estimators['bf_heating_estimator'] = bf_heating_estimator.loc[:, [0]].values
continuum_estimators['stim_recomb_cooling_estimator'] = stim_recomb_cooling_estimator.loc[:, [0]].values
continuum_estimators['coll_deexc_heating_estimator'] = coll_deexc_heating_estimator.loc[:, [0]].values
continuum_estimators['ff_heating_estimator'] = [ff_heating_estimator[0]]

```yml
var: diff
alpha: 1e-5 
alpha_sp: 1e-6 
alpha_stim: 1e-6 
beta_sobolevs: 1e-5 
coll_deexc_coeff: 1e-7 
coll_exc_coeff: 1e-5 
coll_ion: 1e-5, first value should be zero
coll_ion_coeff: 1e-6, first value should be zero
coll_recomb: 1e-5, first value should be zero
coll_recomb_coeff: 1e-6, first value should be zero
collision_rates: 1e-5, diagonal should be negative sum of column, missing lower left triangle
gamma: identical
gamma_vec: identical, first value should be zero
general_level_boltzmann_factor: identical
initial: 1e-5
j_blues: identical
lines_idx: identical
n_e: 1e-9
next_ion_density: 1e-5
phis: 1e-5
previous_electron_densities: 1e-9
previous_ion_number_density: 1e-5
r_lu_index: identical
r_lu_matrix: 1e-9
r_ul_index: identical
r_ul_matrix: 1e-9
rates_matrix: 1e-5
remaining_rates: incorrect fill, missing lower left triangle
t_electrons: identical
x: 1e-6, first value should be zero
```

In [20]:
plasma.update_radiationfield(
            radiation_temp, dilution_factor, j_blues_ctardis["0"],
            config.plasma.nlte, initialize_nlte=False,
            n_e_convergence_threshold=0.05, **continuum_estimators)

In [21]:
plasma.beta_sobolev

0
atomic_number ion_number level_number_lower level_number_upper          
1             0          0                  29                  0.315730
                                            28                  0.289182
                                            27                  0.263190
                                            26                  0.237957
                                            25                  0.213749
...                                                                  ...
                         24                 25                  0.027734
                         25                 26                  0.024914
                         26                 27                  0.022489
                         27                 28                  0.020347
                         28                 29                  0.018466

[435 rows x 1 columns]

In [22]:
plasma.ion_number_density

0
atomic_number ion_number              
1             0           1.541443e+05
              1           2.206764e+09

In [23]:
plasma.level_number_density.head()

0
atomic_number ion_number level_number               
1             0          0             154121.731655
                         1                  1.903318
                         2                  0.278905
                         3                  0.146771
                         4                  0.137661

In [24]:
plasma.electron_densities

0    2.206764e+09
dtype: float64

In [25]:
def iteration(initial, n_e_max, nfev):
    nfev += 1
    n_e_frac = initial[::2]
    link_t_rad_t_electron = initial[1::2]

    print ("Nfev: {} \n".format(nfev))
    print ("link:", link_t_rad_t_electron)

    pl = plasma

    electron_densities = n_e_max * n_e_frac

    plasma.update(previous_ion_number_density=pl.ion_number_density.copy(),
                    previous_electron_densities=electron_densities,
                    previous_beta_sobolev=pl.beta_sobolev.copy(),
                    link_t_rad_t_electron=link_t_rad_t_electron,
                    previous_b=pl.b,
                    previous_t_electrons=pl.t_rad * link_t_rad_t_electron)

    output = np.zeros(2 * len(plasma.fractional_heating))
    frac_e_change = (pl.electron_densities - electron_densities) / electron_densities
    n_e_frac_new = (1 - pl.electron_densities / n_e_max)
    n_e_frac_change = (n_e_frac_new - (1. - n_e_frac)) / (1. - n_e_frac)

    if np.logical_not(np.isfinite(plasma.fractional_heating)).sum() > 0:
        print ("Heating not finite\n")
    if np.logical_not(np.isfinite(frac_e_change)).sum() > 0:
        print ("frac e change not finite\n")

    output[::2] = frac_e_change
    output[1::2] = plasma.fractional_heating
    print ("Frac e change:", frac_e_change)
    print ("n_e_frac_change", n_e_frac_change)
    print ("Heating:", plasma.fractional_heating)
    return output

In [26]:
from scipy.optimize import least_squares as lsq
from scipy.sparse import block_diag

link_t_rad_t_electron_start = plasma.link_t_rad_t_electron
if np.array_equal(
    link_t_rad_t_electron_start, np.ones_like(link_t_rad_t_electron_start)
):
    link_t_rad_t_electron_start = dilution_factor**0.25
    print("Setting initial guess for link from ws:")
    print(link_t_rad_t_electron_start)

n_e_max = (
    (plasma.number_density.multiply(plasma.number_density.index.values, axis=0))
    .sum()
    .values
)
n_e_frac_start = (plasma.electron_densities / n_e_max).values

print("n_e_frac:", n_e_frac_start)

initial = np.zeros(2 * len(link_t_rad_t_electron_start))
initial[::2] = n_e_frac_start
initial[1::2] = link_t_rad_t_electron_start

no_shells = len(radiation_temp)
first_iteration = plasma.gamma is None

nfev = 0
if not first_iteration:
    jac_sparsity = block_diag([np.ones((2, 2))] * no_shells)
    # lbound = [0., 0.35] * no_shells
    t_floor = 1500.0
    link_floor = t_floor / radiation_temp.min()
    print("Floor Link:", link_floor)

    lbound = [0.0, link_floor] * no_shells
    ubound = [1.0, 1.5] * no_shells
    plasma.plasma_converged = False
    x = lsq(
        iteration,
        initial,
        bounds=(lbound, ubound),
        jac_sparsity=jac_sparsity,
        xtol=1e-14,
        ftol=1e-12,
        x_scale="jac",
        verbose=1,
        max_nfev=100,
        method="trf",
        gtol=1e-14,
        args=(n_e_max, nfev,),
    )
    plasma.plasma_converged = True
    final = iteration(x.x, n_e_max, nfev)

    ion_ratio = (
        plasma.ion_number_density.loc[(1, 1)]
        / plasma.ion_number_density.loc[(1, 1)]
    ).values
    print("Ion Ratio:", ion_ratio, ion_ratio**-1)
    print("Plasma Ion Ratio", plasma.ion_ratio)
    ion_ratio_conv = np.fabs(plasma.ion_ratio - ion_ratio**-1) / ion_ratio**-1
    print("Ion Ratio Conv:", ion_ratio_conv)

Setting initial guess for link from ws:
[0.77308588]
n_e_frac: [0.99993015]
Floor Link: 0.1501160051910816
Nfev: 1 

link: [0.77308588]
Frac e change: 0   -0.000024
dtype: float64
n_e_frac_change 0    0.342071
dtype: float64
Heating: [0.00782841]
Nfev: 1 

link: [0.7730859]
Frac e change: 0   -0.000024
dtype: float64
n_e_frac_change 0    0.342071
dtype: float64
Heating: [0.00782841]
Nfev: 1 

link: [0.77308588]
Frac e change: 0   -0.000024
dtype: float64
n_e_frac_change 0    0.342357
dtype: float64
Heating: [0.00782841]
Nfev: 1 

link: [0.81690367]
Frac e change: 0   -9.379911e-07
dtype: float64
n_e_frac_change 0    0.010786
dtype: float64
Heating: [0.00021042]
Nfev: 1 

link: [0.81690369]
Frac e change: 0   -9.379892e-07
dtype: float64
n_e_frac_change 0    0.010786
dtype: float64
Heating: [0.00021042]
Nfev: 1 

link: [0.81690367]
Frac e change: 0   -9.528934e-07
dtype: float64
n_e_frac_change 0    0.010959
dtype: float64
Heating: [0.00021042]
Nfev: 1 

link: [0.81807394]
Frac e change